In [6]:
# import torch and other necessary modules from torch
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score, recall_score, precision_score
...
# import torchvision and other necessary modules from torchvision 
import torchvision
from torchvision import transforms
from torchvision import datasets
from torchvision.transforms import ToTensor

...


# recommended preprocessing steps: resize to square -> convert to tensor -> normalize the image
# if you are resizing, 100 is a good choice otherwise GradeScope will time out
# you could use Compose (https://pytorch.org/vision/stable/generated/torchvision.transforms.Compose.html) from transforms module to handle preprocessing more conveniently
transform = transforms.Compose([
    transforms.Resize((100, 100)),  
    transforms.ToTensor(),  
    transforms.Normalize((0.5,), (0.5,)) 
])


# thanks to torchvision, this is a convenient way to read images from folders directly without writing datasets class yourself (you should know what datasets class is as mentioned in the documentation)
dataset = datasets.ImageFolder(root='./petimages', transform=transform) #./petimages for gradescope #'/content/drive/MyDrive/ASU Spring 2023/CSE 475/Labs/petimages'


# now we need to split the data into training set and evaluation set 
# use 20% of the dataset as test
test_set, train_set = torch.utils.data.random_split(dataset, [int(len(dataset)*0.2), int(len(dataset)*0.8)])

# model hyperparameter
learning_rate = 0.05   #between 0.001 and 0.01
batch_size =  32 #Powers of 2: 16,32,64,128,256,512,1024 : Default is 32
epoch_size = 10

#test_set = torch.utils.data.Subset(dataset, range(n_test))  # take first 10%
#train_set = torch.utils.data.Subset(dataset, range(n_test, len(dataset)))  # take the rest  

# Prepare DataLoader for training set
trainloader = DataLoader(train_set, batch_size = batch_size, shuffle=True)

# Prepare DataLoader for evaluation set
testloader = DataLoader(test_set, batch_size = batch_size, shuffle=False)


# model design goes here
class CNN(nn.Module):

    # there is no "correct" CNN model architecture for this lab, you can start with a naive model as follows:
    # convolution -> relu -> pool -> convolution -> relu -> pool -> convolution -> relu -> pool -> linear -> relu -> linear -> relu -> linear
    # you can try increasing number of convolution layers or try totally different model design
    # convolution: nn.Conv2d (https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)
    # pool: nn.MaxPool2d (https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html)
    # linear: nn.Linear (https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)

    def __init__(self):
        super(CNN,self).__init__()
        #Naive model
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3) #square kernels and equal stride
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2) # pool of square window of size=2, stride=2
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3) #square kernels and equal stride
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) # pool of square window of size=3, stride=2
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3) #square kernels and equal stride
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2) # pool of square window of size=3, stride=2
        self.flat1 = nn.Flatten()
        self.lin1 = nn.Linear(in_features=64*10*10,out_features=256)
        self.relu4 = nn.ReLU()
        self.lin2 = nn.Linear(in_features=256,out_features=128)
        self.relu5 = nn.ReLU()
        self.lin3 = nn.Linear(in_features=128,out_features=2)
    def forward(self, x):          
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.pool3(x)
        x = self.flat1(x)
        x = self.lin1(x)
        x = self.relu4(x)
        x = self.lin2(x)
        x = self.relu5(x)
        x = self.lin3(x)
        return x                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          



device = 'cuda' if torch.cuda.is_available() else 'cpu' # whether your device has GPU
cnn = CNN().to(device) # move the model to GPU
# search in official website for CrossEntropyLoss
criterion = nn.CrossEntropyLoss()
# try Adam optimizer (https://pytorch.org/docs/stable/generated/torch.optim.Adam.html) with learning rate 0.0001, feel free to use other optimizer
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.0001)


# start model training
cnn.train() # turn on train mode, this is a good practice to do
for epoch in range(epoch_size): # begin with trying 10 epochs 

    loss = 0.0 # you can print out average loss per batch every certain batches

    for i, data in enumerate(trainloader, 0):
        # get the inputs and label from dataloader
        inputs, labels = data
        # move tensors to your current device (cpu or gpu)
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients using zero_grad()
        optimizer.zero_grad()
        # forward -> compute loss -> backward propogation -> optimize (see tutorial mentioned in main documentation)
        outputs = cnn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print some statistics
        loss +=  loss.item()# add loss for current batch 
        if i % 100 == 99:    # print out average loss every 100 batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {loss / 100:.3f}')
            loss = 0.0

print('Finished Training')


# evaluation on evaluation set
ground_truth = []
prediction = []
cnn.eval() # turn on evaluation model, also a good practice to do
with torch.no_grad(): # since we're not training, we don't need to calculate the gradients for our outputs, so turn on no_grad mode
    for data in testloader:
        inputs, labels = data
        inputs = inputs.to(device)
        ground_truth += labels.tolist() # convert labels to list and append to ground_truth
        # calculate outputs by running inputs through the network
        outputs = cnn(inputs)
        # the class with the highest logic is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        prediction += predicted.tolist()  # convert predicted to list and append to prediction


# GradeScope is chekcing for these three variables, you can use sklearn to calculate the scores
accuracy = accuracy_score(ground_truth, prediction)
recall = recall_score(ground_truth, prediction)
precision = precision_score(ground_truth, prediction)








[1,   100] loss: 0.014
[2,   100] loss: 0.013
[3,   100] loss: 0.011
[4,   100] loss: 0.009
[5,   100] loss: 0.012
[6,   100] loss: 0.011
[7,   100] loss: 0.009
[8,   100] loss: 0.010
[9,   100] loss: 0.011
[10,   100] loss: 0.010
Finished Training
